In [37]:
import requests
import json
from bs4 import BeautifulSoup

In [38]:
d = ""
with open("allDataP1-P4IDS.json") as file:
    d = json.loads(file.read())

In [39]:
class Item:
    def __init__(self, id, npcID, npcName, kill, quest, questID, recipe, recipeID, dropChance, zone):
        self.id = id
        self.Obtain = {}
        self.Obtain["NpcID"] = npcID
        self.Obtain["NpcName"] = npcName
        self.Obtain["Kill"] = kill
        self.Obtain["Quest"] = quest
        self.Obtain["QuestID"] = questID
        self.Obtain["Recipe"] = recipe
        self.Obtain["RecipeID"] = recipeID
        self.Obtain["DropChance"] = dropChance
        self.Obtain["Zone"] = zone
    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)

In [44]:
def get_region(content, start, end = None):
    idx_s = content.index(start)
    idx_e = len(content)
    if end:
        idx_e = content.index(end, idx_s)
    return content[idx_s + len(start): idx_e]

In [115]:

URL = 'https://classic.wowhead.com/item='
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

main = soup.find(id='main-contents')
scripts = main.findAll('script')


QuestCaregoryUrl = 'https://classic.wowhead.com/quests='
ZoneUrl = 'https://classic.wowhead.com/zone='

SCRIPT2_START = "var tabsRelated = new Tabs({parent: WH.ge('jkbfksdbl4'), trackable: 'Item'});"
DATA2_START = 'data:'

#for child in scripts:
#    content = ''.join(child.contents).replace('\n   ', '').strip()
#    if content.startswith(SCRIPT2_START):
#        data2_container = content
#        break
#r0 = get_region(data2_container, DATA2_START)
#r1 = get_region(r0, ' ', ',\n});')
#print(json.loads(r1))


#    var tabsRelated = new Tabs({parent: WH.ge('jkbfksdbl4'), trackable: 'Item'});
#new Listview({
#    template: 'object',
#    id: 'contained-in-object',
#    name: WH.TERMS.containedin,
#    tabs: 'tabsRelated',
#    parent: 'lkljbjkb574',
#    hiddenCols: ['type'],
#    extraCols: ['count', 'percent', 'popularity'],
#    sort: ['-percent', 'name'],
#    computeDataFunc: Listview.funcBox.initLootTable,
#        data: [{"id":179703,"location":[2717],"name":"Cache of the Firelord","type":3,"count":99,"outof":645,"pctstack":"","popularity":144}],
#});
#


#####           GOTTA REDO HOW YOU GET ACQUISITION METHOD!          #####

def GetZone(zoneID):
        zUrl = ZoneUrl + str(zoneID)
        pageZone = requests.get(zUrl)
        soupZone = BeautifulSoup(pageZone.content, 'html.parser')
        zoneNameH1 = soupZone.find('h1', class_='heading-size-1')
        zoneName = zoneNameH1.text
        return zoneName

def GetCategory(categoryID, category2ID):
        cUrl = QuestCaregoryUrl + str(category2ID) + "." + str(categoryID)
        pageCat = requests.get(cUrl)
        soupCat = BeautifulSoup(pageCat.content, 'html.parser')
        CatNameH1 = soupCat.find('h1', class_='heading-size-1')
        zoneName = CatNameH1.text.replace(" Quests", "")
        return zoneName

def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

def GetItem(itemID):
    if (itemID == 0):
        return Item(itemID, 0, "", False, False, 0, False, 0, 0, "").toJson()
    data2_container = None
    print("Scraping item", itemID)
    o = Item(itemID, 0, "Unknown acquisition method", False, False, 0, False, 0, 0, "")
    url = URL + str(itemID)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    main = soup.find(id='main-contents')
    scripts = main.findAll('script')

    for child in scripts:
        content = ''.join(child.contents).replace('\n   ', '').strip()
        if content.startswith(SCRIPT2_START):
            data2_container = content
            break
    r0 = get_region(data2_container, DATA2_START)
    r1 = get_region(r0, ' ', ',\n});')
    
    data = json.loads(r1)
    usefulData = data[0]
    zoneName = "Unknown"

    ##  REDO - IT LITERALLY TELLS YOU WHAT THE ACQUISITION METHOD IS YOU DUMBASS
    if ('classification' in usefulData):
        dropChance = 0
        for i in data:
            if ('count' in i):
                if (i['count'] >= 0 and i['outof'] > 0):
                    if (truncate(int(i['count'])/int(i['outof']) * 100, 1) > dropChance):
                        usefulData = i
                        dropChance = truncate(int(usefulData['count'])/int(usefulData['outof']) * 100, 1)
        if ('location' in usefulData):
            zoneName = GetZone(usefulData['location'][0])
        if ('cost' in usefulData):
            dropChance = 100
            usefulData['name'] = usefulData['name'] + " (Purchase)"
        o = Item(itemID, usefulData['id'], usefulData['name'], True, False, 0, False, 0, dropChance, zoneName)
    elif ('category' in usefulData):
        if ('category' in usefulData and 'category2' in usefulData):
            zoneName = GetCategory(usefulData['category'], usefulData['category2'])
        o = Item(itemID, 0, "", False, True, usefulData['id'], False, 0, 0, zoneName)
    elif ('cat' in usefulData):
        o = Item(itemID, 0, "", False, False, 0, True, usefulData['id'], 0, "")
    return o.toJson()


In [110]:
print(GetItem(12553))

Scraping item 12553
{'boss': 1, 'classification': 1, 'id': 8929, 'location': [1584], 'maxlevel': 58, 'minlevel': 58, 'name': 'Princess Moira Bronzebeard', 'react': [-1, -1], 'tag': 'Princess of Ironforge', 'type': 7, 'count': 863, 'outof': 3791, 'personal_loot': 0, 'pctstack': '', 'popularity': 362}
{"id": 12553, "Obtain": {"NpcID": 8929, "NpcName": "Princess Moira Bronzebeard", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 22.7, "Zone": "Blackrock Depths"}}


In [116]:
nd = {}
for k, v in d.items():
    print("Scraping class", k) # Class
    nd[k] = {}
    for k2, v2 in v.items():
        for k3, v3 in v2.items():
            ns = k2 + "-" + k3
            print("Scraping Set", ns) # Phase
            nd[k][ns] = {}
            nd[k][ns]["Head"] = GetItem(d[k][k2][k3]["Head"]["itemID"])
            nd[k][ns]["Neck"] = GetItem(d[k][k2][k3]["Neck"]["itemID"])
            nd[k][ns]["Shoulder"] = GetItem(d[k][k2][k3]["Shoulder"]["itemID"])
            nd[k][ns]["Back"] = GetItem(d[k][k2][k3]["Cloak"]["itemID"])
            nd[k][ns]["Chest"] = GetItem(d[k][k2][k3]["Chest"]["itemID"])
            nd[k][ns]["Wrists"] = GetItem(d[k][k2][k3]["Wrist"]["itemID"])
            nd[k][ns]["Hands"] = GetItem(d[k][k2][k3]["Gloves"]["itemID"])
            nd[k][ns]["Waist"] = GetItem(d[k][k2][k3]["Waist"]["itemID"])
            nd[k][ns]["Legs"] = GetItem(d[k][k2][k3]["Legs"]["itemID"])
            nd[k][ns]["Feet"] = GetItem(d[k][k2][k3]["Boots"]["itemID"])
            nd[k][ns]["Finger"] = GetItem(d[k][k2][k3]["Ring1"]["itemID"])
            nd[k][ns]["RFinger"] = GetItem(d[k][k2][k3]["Ring2"]["itemID"])
            nd[k][ns]["Trinket"] = GetItem(d[k][k2][k3]["Trinket1"]["itemID"])
            nd[k][ns]["RTrinket"] = GetItem(d[k][k2][k3]["Trinket2"]["itemID"])
            nd[k][ns]["MainHand"] = GetItem(d[k][k2][k3]["MainHand"]["itemID"])
            nd[k][ns]["SecondaryHand"] = GetItem(d[k][k2][k3]["OffHand"]["itemID"])
            nd[k][ns]["Relic"] = GetItem(d[k][k2][k3]["Ranged"]["itemID"])
print("---Done with scrape---")

Scraping class Druid
Scraping Set FeralTank-P1
Scraping item 13404
Scraping item 12927
Scraping item 10783
Scraping item 13397
Scraping item 15064
Scraping item 12966
Scraping item 13258
Scraping item 14502
Scraping item 12963
Scraping item 18716
Scraping item 15855
Scraping item 11669
Scraping item 13966
Scraping item 11811
Scraping item 868
Scraping item 13385
Scraping item 23198
Scraping Set FeralTank-P2
Scraping item 14539
Scraping item 17065
Scraping item 19139
Scraping item 17107
Scraping item 15064
Scraping item 12966
Scraping item 13258
Scraping item 19149
Scraping item 11821
Scraping item 16711
Scraping item 15855
Scraping item 18879
Scraping item 13966
Scraping item 11811
Scraping item 868
Scraping item 13385
Scraping item 23198
Scraping Set FeralTank-P3
Scraping item 20623
Scraping item 19383
Scraping item 19389
Scraping item 19386
Scraping item 19405
Scraping item 11766
Scraping item 13258
Scraping item 19163
Scraping item 22740
Scraping item 19381
Scraping item 17063
Scrap

ValueError: substring not found

In [40]:
URL = 'https://classic.wowhead.com/item='
QuestCaregoryUrl = 'https://classic.wowhead.com/quests=0.'
ZoneUrl = 'https://classic.wowhead.com/zone='
SCRIPT_START = 'WH.Gatherer.addData('
DATA_START = '$.extend('



def GetItem(itemID):
    if (itemID == 0):
        return Item(itemID, 0, "", False, False, 0, False, 0, 0, "").toJson()

    print("Scraping Item", itemID)
    url = URL+str(itemID)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    main = soup.find(id='main-contents')
    scripts = main.findAll('script')
    data_container = None
    o = Item(itemID, 0, "Unknown acquisition method", False, False, 0, False, 0, 0, "")

    #Find script tag
    for child in scripts:
        content = ''.join(child.contents).replace('\n    ', '').strip()
        if content.startswith(SCRIPT_START):
            data_container = content
            break
    r0 = get_region(data_container, DATA_START)
    r1 = get_region(r0, ', ', ');')
    data = json.loads(r1)
    if ('sourcemore' in data):
        usefulData = data['sourcemore'][0]
        if ('t' in usefulData):
            if usefulData['t'] == 1: # Kill
                ZoneName = "Unknown Location"
                if ('z' in usefulData):
                    zUrl = ZoneUrl + str(usefulData['z'])
                    pageZone = requests.get(zUrl)
                    soupZone = BeautifulSoup(pageZone.content, 'html.parser')
                    ZoneNameH1 = soupZone.find('h1', class_='heading-size-1')
                    ZoneName = ZoneNameH1.text
                dropDiv = soup.find('div', class_="whtt-dropchance")
                dropChance = 0
                if (dropDiv is not None):
                    dropChance = dropDiv.text.replace("Drop Chance: ", "").replace("%", "")
                o = Item(itemID, usefulData['ti'], usefulData['n'], True, False, 0, False, 0, dropChance, ZoneName)
            elif usefulData['t'] == 5: # Quest
                cUrl = QuestCaregoryUrl + str(usefulData['c'])
                pageCat = requests.get(cUrl)
                soupCat = BeautifulSoup(pageCat.content, 'html.parser')
                CatNameH1 = soupCat.find('h1', class_='heading-size-1')
                ZoneName = CatNameH1.text.replace(" Quests", "")
                o = Item(itemID, 0, "", False, True, usefulData['ti'], False, 0, 0, ZoneName)
            elif usefulData['t'] == 6: # Recipe
                o = Item(itemID, 0, "", False, False, 0, True, usefulData['ti'], 0, "")
        else:
            if ('z' in usefulData):
                zUrl = ZoneUrl + str(usefulData['z'])
                pageZone = requests.get(zUrl)
                soupZone = BeautifulSoup(pageZone.content, 'html.parser')
                ZoneNameH1 = soupZone.find('h1', class_='heading-size-1')
                ZoneName = ZoneNameH1.text
                print("---------No type")
                o = Item(itemID, 0, "Unknown acquisition method", False, False, 0, False, 0, 0, ZoneName)
    else:
        print("---------No Sourcemore")
    return o.toJson()


In [117]:
print(json.dumps(nd).replace('"{', '{').replace('}"', '}').replace('\\', ''))

{"Druid": {"FeralTank-P1": {"Head": {"id": 13404, "Obtain": {"NpcID": 10516, "NpcName": "The Unforgiven", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 20.8, "Zone": "Stratholme"}}, "Neck": {"id": 12927, "Obtain": {"NpcID": 9816, "NpcName": "Pyroguard Emberseer", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 11.6, "Zone": "Blackrock Spire"}}, "Shoulder": {"id": 10783, "Obtain": {"NpcID": 5716, "NpcName": "Zul'Lor", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 5.0, "Zone": "The Temple of Atal'Hakkar"}}, "Back": {"id": 13397, "Obtain": {"NpcID": 10809, "NpcName": "Stonespine", "Kill": true, "Quest": false, "QuestID": 0, "Recipe": false, "RecipeID": 0, "DropChance": 31.1, "Zone": "Stratholme"}}, "Chest": {"id": 15064, "Obtain": {"NpcID": 0, "NpcName": "", "Kill": false, "Quest": false, "QuestID": 0, "Recipe": true, "RecipeID": 19068, "DropChance": 0, "Zone":

In [118]:
newFile = open("BiSDataP1-4", "w")
newFile.write(json.dumps(nd).replace('"{', '{').replace('}"', '}').replace('\\', ''))
newFile.close()